In [17]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.model_selection import train_test_split
import torch.nn.functional as F


np.random.seed(42)


x1 = np.random.rand(1000) * 2 - 1
x2 = np.random.rand(1000) * 2 - 1
y = np.exp(x1) + 0.1 * x2/4  # Adjusted relationship

X = np.column_stack((x1, y))
targets = x2[:,None]

X_train, X_test, y_train, y_test = train_test_split(X, targets, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(2, 30)  
        self.fc2 = nn.Linear(30, 1)  

    def forward(self, x):

        x = F.silu(self.fc1(x)) 
        x = self.fc2(x)  
        return x


model = Net()
criterion = nn.MSELoss()  # Mean squared error loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Adam optimizer


def train_model(num_epochs):
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        #loss = criterion(outputs.view(-1), y_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')


train_model(1000)


model.eval()
with torch.no_grad():
    predicted_x2 = model(X_test_tensor).view(-1)
    test_loss = criterion(predicted_x2, y_test_tensor)
    print(f'Test Loss: {test_loss.item()}')


Epoch 0, Loss: 0.3740602433681488
Epoch 100, Loss: 0.32918334007263184
Epoch 200, Loss: 0.22825703024864197
Epoch 300, Loss: 0.12442159652709961
Epoch 400, Loss: 0.07359926402568817
Epoch 500, Loss: 0.04953790828585625
Epoch 600, Loss: 0.03671818971633911
Epoch 700, Loss: 0.030293302610516548
Epoch 800, Loss: 0.027120264247059822
Epoch 900, Loss: 0.024428896605968475
Test Loss: 0.6166499853134155


In [18]:
import plotly.graph_objects as go
import numpy as np

actual_x2 = y_test_tensor.numpy()
predicted_x2 = predicted_x2.numpy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=actual_x2[:,0], y=predicted_x2, mode='markers', name='Prediction'))
fig.add_trace(go.Scatter(x=actual_x2[:,0], y=actual_x2[:,0], mode='lines', name='Ideal Fit', line=dict(color='red')))

fig.update_layout(
    title="Actual x2 vs. Predicted x2",

    xaxis=dict(
        title='Actual x2',
        # type='log',
        title_font=dict(size=18),
        tickfont=dict(size=18),
        showgrid=True,
        gridcolor='lightgrey'
    ),

    yaxis=dict(
        title="Predicted x2",
        # type='log',
        title_font=dict(size=18),
        tickfont=dict(size=18),
        showgrid=True,
        gridcolor='lightgrey'
    ),

    legend_title="Legend",
    width=900, height=800,
    title_x=0.5,
    title_y=0.95,
    template='plotly_white'
)

fig.show()



In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # Increasing the complexity of the network
        self.fc1 = nn.Linear(2, 50)  # First layer increased to 50 neurons
        self.fc2 = nn.Linear(50, 100)  # Second layer with 100 neurons
        self.fc3 = nn.Linear(100, 150)  # Adding a third layer with 150 neurons
        self.fc4 = nn.Linear(150, 100)  # Fourth layer with 100 neurons
        self.fc5 = nn.Linear(100, 50)  # Fifth layer stepping down to 50 neurons
        self.fc6 = nn.Linear(50, 1)    # Final layer outputting to 1 neuron

    def forward(self, x):
        x = F.silu(self.fc1(x))  # Using SiLU activation function
        x = F.silu(self.fc2(x))
        x = F.silu(self.fc3(x))
        x = F.silu(self.fc4(x))
        x = F.silu(self.fc5(x))
        x = self.fc6(x)
        return x

# Everything else remains the same
model = Net()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train_model(num_epochs):
    for epoch in range(num_epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train_tensor)
        loss = criterion(outputs, y_train_tensor)
        loss.backward()
        optimizer.step()

        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss.item()}')

train_model(1000)

model.eval()
with torch.no_grad():
    predicted_x2 = model(X_test_tensor).view(-1)
    test_loss = criterion(predicted_x2, y_test_tensor)
    print(f'Test Loss: {test_loss.item()}')


Epoch 0, Loss: 0.3493354022502899
Epoch 100, Loss: 0.2920891046524048
Epoch 200, Loss: 0.3343275487422943
Epoch 300, Loss: 0.33489990234375
Epoch 400, Loss: 0.21794810891151428
Epoch 500, Loss: 0.3378247022628784
Epoch 600, Loss: 0.39974838495254517
Epoch 700, Loss: 0.29045620560646057
Epoch 800, Loss: 0.34050655364990234
Epoch 900, Loss: 0.3481537699699402
Test Loss: 0.3426372706890106
